In [3]:

import pandas as pd
import numpy as np
import os
import joblib
from tqdm import notebook, tqdm
from scipy.sparse import coo_matrix
from functools import partial

# multiprocessing to speedup matrix extraction
import multiprocessing



In [4]:
# preprocess data csv
read_col = ['Anon Student Id', 'Problem Name', 'KC(SubSkills)', 'Correct First Attempt']
ddf = pd.read_csv("bridge_to_algebra_2008_2009_train.txt", low_memory=False, error_bad_lines=False, sep="\t")
ddf

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(SubSkills),Opportunity(SubSkills),KC(KTracedSkills),Opportunity(KTracedSkills)
0,1,stu_cc883f541b,"Unit WHOLE-NUMBER-OPERATIONS-1, Section WHOLE-...",WHOLENUMOPS1-083,1,Operation,2009-02-03 14:11:58.0,2009-02-03 14:14:19.0,2009-02-03 14:14:19.0,2009-02-03 14:14:19.0,...,141.000,NaN,1,0,0,1,Identify operator,1,Identify operator-1,1
1,2,stu_cc883f541b,"Unit WHOLE-NUMBER-OPERATIONS-1, Section WHOLE-...",WHOLENUMOPS1-083,2,Digit1-1,2009-04-16 14:24:41.0,2009-04-16 14:27:09.0,2009-04-16 14:27:15.0,2009-04-16 14:27:15.0,...,NaN,154.000,0,2,0,1,NaN,NaN,NaN,NaN
2,3,stu_cc883f541b,"Unit WHOLE-NUMBER-OPERATIONS-1, Section WHOLE-...",WHOLENUMOPS1-083,2,Digit2-1,2009-04-16 14:27:51.0,2009-04-16 14:28:01.0,2009-04-16 14:28:51.0,2009-04-16 14:28:51.0,...,NaN,59.998,0,11,0,1,NaN,NaN,NaN,NaN
3,4,stu_cc883f541b,"Unit WHOLE-NUMBER-OPERATIONS-1, Section WHOLE-...",WHOLENUMOPS1-083,2,Digit2-2,2009-04-16 14:28:51.0,2009-04-16 14:28:54.0,2009-04-16 14:29:09.0,2009-04-16 14:29:09.0,...,NaN,18.000,0,9,0,1,NaN,NaN,NaN,NaN
4,5,stu_cc883f541b,"Unit WHOLE-NUMBER-OPERATIONS-1, Section WHOLE-...",WHOLENUMOPS1-083,2,Digit2-3,2009-04-16 14:29:09.0,2009-04-16 14:29:11.0,2009-04-16 14:29:11.0,2009-04-16 14:29:11.0,...,2.000,NaN,1,0,0,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20012493,20012494,stu_74a2bf0e60,"Unit LCM, Section lcm-2",LCM2-146,1,CommonMultiple1,2008-11-06 15:30:56.0,2008-11-06 15:31:00.0,2008-11-06 15:31:00.0,2008-11-06 15:31:00.0,...,3.250,NaN,1,0,0,1,Identify LCM,17,Identify LCM-1,17
20012494,20012495,stu_74a2bf0e60,"Unit LCM, Section lcm-2",LCM2-079,1,Product,2008-11-06 15:31:13.0,2008-11-06 15:32:31.0,2008-11-06 15:32:42.0,2008-11-06 15:32:42.0,...,NaN,89.453,0,1,1,1,Calculate product of two numbers,23,NaN,NaN
20012495,20012496,stu_74a2bf0e60,"Unit LCM, Section lcm-2",LCM2-079,1,CommonMultiple1,2008-11-06 15:32:42.0,2008-11-06 15:33:02.0,2008-11-06 15:33:22.0,2008-11-06 15:33:22.0,...,NaN,39.484,0,2,2,1,Identify LCM,18,Identify LCM-1,18
20012496,20012497,stu_74a2bf0e60,"Unit LCM, Section lcm-2",LCM2-041,1,Product,2008-11-06 15:33:26.0,2008-11-06 15:34:06.0,2008-11-06 15:34:19.0,2008-11-06 15:34:19.0,...,NaN,52.390,0,1,1,1,Calculate product of two numbers,24,NaN,NaN


In [29]:
set(ddf['KC(SubSkills)'])


{nan,
 'Calculate fraction from given fractional percent',
 'Using simple numbers~~Using large numbers~~Write Expression, mx+b~~Write expression, positive intercept~~Write expression, any form~~Write expression, positive slope',
 'Identify negative integer from number line~~Identify integer from symmetrical number line',
 'Enter ratio quantity to right of "to"',
 'Work with cylinder in standard position~~Find cylinder volume out of context',
 'Using large numbers~~Using difficult numbers~~Find Y, negative slope~~Find Y, any form',
 'Find square-pyramid volume in context~~Work with square-pyramid in standard position',
 'Measure angle opening right not on horizontal~~Measure acute angle',
 '[SkillRule: Simplify across fractions; {Simplify Across Fractions, both numers and denoms; Simplify Across Fractions, numer-1 and denom-2; Simplify Across Fractions, denom-1 and numer-2}]',
 'Identify whole number lower bound',
 '[SkillRule: Invert divisor and multiply; Invert and Multiply]~~[SkillRu

In [ ]:
ddf.columns
ddf.info()

In [5]:
df = ddf[read_col]

df.rename(columns={read_col[0]: 'user_id', read_col[1]: 'problem_id', read_col[2]: 'skill_id',
                    read_col[3]: 'correct'}, inplace=True)
df

/home/cenjw/anaconda3/envs/dkt/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,user_id,problem_id,skill_id,correct
0,stu_cc883f541b,WHOLENUMOPS1-083,Identify operator,1
1,stu_cc883f541b,WHOLENUMOPS1-083,NaN,0
2,stu_cc883f541b,WHOLENUMOPS1-083,NaN,0
3,stu_cc883f541b,WHOLENUMOPS1-083,NaN,0
4,stu_cc883f541b,WHOLENUMOPS1-083,NaN,1
...,...,...,...,...
20012493,stu_74a2bf0e60,LCM2-146,Identify LCM,1
20012494,stu_74a2bf0e60,LCM2-079,Calculate product of two numbers,0
20012495,stu_74a2bf0e60,LCM2-079,Identify LCM,0
20012496,stu_74a2bf0e60,LCM2-041,Calculate product of two numbers,0


In [6]:
df.dropna(inplace=True)
df

/tmp/ipykernel_205214/955798070.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


,user_id,problem_id,skill_id,correct
0,stu_cc883f541b,WHOLENUMOPS1-083,Identify operator,1
15,stu_cc883f541b,WHOLENUMOPS1-083,Enter answer digit -- DON'T TRACK ME~~Enter su...,1
16,stu_cc883f541b,WHOLENUMOPS1-083,Enter answer digit -- DON'T TRACK ME~~Enter su...,1
17,stu_cc883f541b,WHOLENUMOPS1-083,Enter answer digit -- DON'T TRACK ME~~Enter su...,1
18,stu_cc883f541b,WHOLENUMOPS1-083,Enter answer digit -- DON'T TRACK ME~~Enter su...,1
...,...,...,...,...
20012493,stu_74a2bf0e60,LCM2-146,Identify LCM,1
20012494,stu_74a2bf0e60,LCM2-079,Calculate product of two numbers,0
20012495,stu_74a2bf0e60,LCM2-079,Identify LCM,0
20012496,stu_74a2bf0e60,LCM2-041,Calculate product of two numbers,0


In [7]:
df.user_id,_ = pd.factorize(df.user_id, sort=False)
df.problem_id,_ = pd.factorize(df.problem_id, sort=False)
df.skill_id,_= pd.factorize(df.skill_id, sort=False)

/home/cenjw/anaconda3/envs/dkt/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
df

,user_id,problem_id,skill_id,correct
0,0,0,0,1
15,0,0,1,1
16,0,0,1,1
17,0,0,1,1
18,0,0,1,1
...,...,...,...,...
20012493,5994,14463,151,1
20012494,5994,14625,147,0
20012495,5994,14625,151,0
20012496,5994,18082,147,0


In [8]:
# delete empty skill_id
df = df.dropna(subset=['skill_id'])
df = df[~df['skill_id'].isin(['noskill'])]
df.skill_id = df.skill_id.astype('int')
print('After removing empty skill_id, records number %d' % len(df))

min_inter_num = 10
users = df.groupby(['user_id'], as_index=True)
delete_users = []
for u in users:
    if len(u[1]) < min_inter_num:
        delete_users.append(u[0])
print('deleted user number based min-inters %d' % len(delete_users))
df = df[~df['user_id'].isin(delete_users)]
df = df[[ 'user_id', 'problem_id', 'skill_id', 'correct']]
print('After deleting some users, records number %d' % len(df))

After removing empty skill_id, records number 12350449
deleted user number based min-inters 250
After deleting some users, records number 12349223


In [10]:
df

,user_id,problem_id,skill_id,correct
68,3,2,0,1
69,3,2,2,1
70,3,2,2,1
71,3,2,2,1
72,3,2,2,1
...,...,...,...,...
20012493,5994,14463,151,1
20012494,5994,14625,147,0
20012495,5994,14625,151,0
20012496,5994,18082,147,0


In [19]:
import random
users = df['user_id'].unique().tolist()[50:]

tdf = df[~df['user_id'].isin(users)]
len(users),tdf

(5695,
         user_id  problem_id  skill_id  correct
 68            3           2         0        1
 69            3           2         2        1
 70            3           2         2        1
 71            3           2         2        1
 72            3           2         2        1
 ...         ...         ...       ...      ...
 121244       52        6490       224        1
 121245       52        6490       225        1
 121246       52        6137       227        0
 121247       52        6137       229        0
 121250       52        6137       228        1
 
 [79727 rows x 4 columns])

In [20]:
skill_df = tdf[['skill_id', 'problem_id']].groupby(['skill_id'], as_index=True).apply(lambda r: np.array(list(set(r['problem_id'].values))))
# skill_df  1,"[96, 97, 98, 99, 100, 101, 102, 103, 104, 92, 93, 94, 95]"
joblib.dump(skill_df, './kddcup/skill_prob.pkl.zip')

user_prob = tdf[['user_id', 'problem_id', 'correct']].groupby(['user_id', 'problem_id'])['correct'].agg('mean')
skill_df


skill_id
0      [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 5374, ...
1      [0, 3, 5, 7, 9, 11, 4624, 604, 606, 608, 609, ...
2      [2, 4, 6, 8, 10, 620, 621, 624, 5710, 5712, 62...
3      [2, 4, 6, 8, 5712, 607, 620, 623, 112, 113, 11...
4      [10, 12, 625, 626, 627, 628, 603, 605, 629, 60...
                             ...                        
767                                               [6408]
768                                               [1418]
769                                         [6427, 6431]
770                                               [6429]
771                                               [6447]
Length: 772, dtype: object

In [21]:
def co_acc_probs(user_prob, prob1, prob2):
    count = 0
    agg = 0
    for user, prob in user_prob.index:
        if prob == prob1:
            if prob2 in user_prob[user].index:
                count += 1
                agg += user_prob[user][prob1] * user_prob[user][prob2]
#                 print('user {} answered two questions, with {} answered {}, and {} answered {}'
#                   .format(user, prob1, user_prob[user][prob1], prob2, user_prob[user][prob2]))
    if count == 0:
        return -1
    else:
        return agg/count


def co_acc_skills(user_skill, skill1, skill2):
    count = 0
    agg = 0
    for user, skill in user_skill.index:
        if skill1 == skill:
            if skill2 in user_skill[user].index:
                count += 1
                agg += user_skill[user][skill1] * user_skill[user][skill2]
    if count == 0:
        return -1
    else:
        return agg/count

In [25]:
cores = multiprocessing.cpu_count()
cores = 90
pool = multiprocessing.Pool(processes=cores)

In [ ]:

# e2e matrix
skill_mats = []
e2e = partial(co_acc_probs, user_prob)
cnt = 1
for skill in tqdm(skill_df.index):
    if cnt % 50 == 0:
        print('processing skill: ', skill)
    cnt+=1
    skill_probs = skill_df[skill] #[1, 87,99 ,77]
    row = []
    col = []

    args = ((prob1, prob2) for i, prob1 in enumerate(skill_probs) for prob2 in skill_probs[i:])
    # 并行处理
    val = pool.starmap(e2e, args)
    print('matrix size is: ', len(val))
    for i in range(len(skill_probs)):
        for j in range(i, len(skill_probs)):
            row.append(i)
            col.append(j)
    assert(len(val)==len(row))
    mat = coo_matrix((val, (row, col)), shape=(len(skill_probs), len(skill_probs)))
    skill_mats.append(mat)
pool.close()
joblib.dump(skill_mats, './kddcup/para_e2e.pkl.zip')

In [26]:
skill_acc = tdf[['skill_id', 'correct']].groupby('skill_id')['correct'].agg('mean')
user_skill = tdf[['user_id', 'skill_id', 'correct']].groupby(['user_id', 'skill_id'])['correct'].agg('mean')

for user, skill in tqdm(user_skill.index):
    if user_skill[user][skill] >= skill_acc[skill]:
        user_skill[user][skill] = 1
    else:
        user_skill[user][skill] = 0

# c2c matrix
row = []
col = []
val = []
print('Extracting c2c matrix...')
arg_skills = ((skill1, skill2) for (i, skill1) in enumerate(skill_df.index) for skill2 in skill_df.index[i:])
c2c = partial(co_acc_skills, user_skill)
val = pool.starmap(c2c, arg_skills)
for i in tqdm(range(len(skill_df.index))):
    for j in range(i, len(skill_df.index)):
        row.append(i)
        col.append(j)
mat_skill = coo_matrix((val, (row, col)), shape=(len(skill_df.index), len(skill_df.index)))
joblib.dump(mat_skill, './kddcup/c2c.pkl.zip')
# print(mat_skill)

100%|██████████| 7579/7579 [00:03<00:00, 1926.37it/s]


Extracting c2c matrix...


100%|██████████| 772/772 [00:00<00:00, 10345.02it/s]


['./kddcup/c2c.pkl.zip']